# Summer School: Digitale Methoden der Zeitungsanalyse

## Teil 3: Analysieren und Visualisieren der Texte der Alto-Dateien: 

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Biblitoheken importiert werden.

- `pandas`: Bibliothek zur Datenanalyse.
- `string`: Funktionen zur Bearbeitung von Zeichenketten
- `nltk` (Natural Language Toolkit): zur Verarbeitung natürlicher Sprache.
- `nltk.corpus`: Liste häufiger Wörter, die bei der Textanalyse oft ignoriert werden sollten
- `collections`: Zum Speichern von Wörtern, Wörterbüchern und zum Zählen von Elementen.
- `matplotlib` : Zum erstellen von Visualisierungen
- `numpy` : Bibliothek für wissenschaftliches Rechnen, Datenanalysen und numerische Berechnungen
- `WorldCloud`: Bibliothek zur Erstellung von WordClouds 

In [ ]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud
import plotly.graph_objects as go

#### Laden der zuvor gespeicherten Texte: 

In [ ]:
df = pd.read_csv("otra_alemania_content.csv", encoding = "UTF-8")
df

## Häufigkeitsanalyse

In [ ]:
# Häufigkeitsanalyse der häufigsten Wörter
# Textinhalte aus DataFrame sammeln
content = df['text'].tolist()
content = [text.lower() for text in content]  # Kleinbuchstaben zur Vereinheitlichung
print(content[0])

In [ ]:
# Entfernen von Satzzeichen und Tokenisierung
listofthings = []
for entry in content:
    for c in string.punctuation:
        entry = entry.replace(c, " ")
    words = entry.split()
    listofthings.extend(words)

In [ ]:
print(listofthings)

In [ ]:
# Entfernen von Zahlen (bspw. Seitenzahlen, andere Zahlen)
words = [word for word in listofthings if not word.isdecimal()]
print(words)

#### Entfernen von Stopwörtern:

In [ ]:
# Importieren der zu entfernenden Stopwörter
nltk.download('stopwords')
stopger = stopwords.words('german')

# Hinzufügen weiterer Stopwörter
newStopwords = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'vgl', '\x97', '•', '■', 'v',
                'beim', 'de','—','ge','la','be','en','que','el','ten','ver','gen','sei','nen','del','nen', 'se','schen','un','land','te','ei','aires',
                'las', 'los', '«']

#Erweitern der heruntergeladenen Stopwörter mit den selbst definierten "newStopwords":
stopger.extend(newStopwords)

In [ ]:
#print(stopger)

In [ ]:
# Entfernen der Stopwörter aus der Gesamtwortliste: 
tokens_without_sw = [word for word in words if word not in stopger]
print(tokens_without_sw)

In [ ]:
# Zählen der Häufigkeit der verbleibenden Wörter
counts = Counter(tokens_without_sw)

# Top 20 häufigste Wörter ausgeben
top_20_words = counts.most_common(20)
top_150_words = counts.most_common(150)
print(top_20_words)

## Visualisierungen erstellen

#### 1. Kreisdiagramm:

In [ ]:
# Funktion zum Erstellen eines Kreisdiagramms
def create_circle(word_count):
    labels = [word[0] for word in word_count]
    sizes = [word[1] for word in word_count]
    colors = [plt.cm.Spectral(i/float(len(labels))) for i in range(len(labels))]

    plt.figure(figsize=(10, 8))
    plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
    plt.axis('equal')  # Kreisdiagramm rund machen
    plt.title('Top 20 häufigste Wörter')
    plt.show()

# Kreisdiagramm erstellen
create_circle(top_20_words)

#### 2. Balkendiagramm: 

In [ ]:
# Funktion zum Erstellen eines Balkendiagramms
def create_bar_chart(word_count):
    labels = [word[0] for word in word_count]
    sizes = [word[1] for word in word_count]
    
    x = np.arange(len(labels))  # Die Label-Positionen
    width = 0.75  # Die Breite der Balken

    fig, ax = plt.subplots(figsize=(12, 8))
    rects = ax.bar(x, sizes, width)

    # Labels hinzufügen, etc.
    ax.set_ylabel('Häufigkeit')
    ax.set_title('Top 20 häufigste Wörter')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    
    # Balken beschriften
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 Punkte vertikal versetzt
                    textcoords="offset points",
                    ha='center', va='bottom')
    
    return fig, ax
    #plt.show()

# Balkendiagramm erstellen
fig, ax = create_bar_chart(top_20_words)

In [ ]:
# Diagramm als Bild speichern
fig.savefig('plot.jpg', format='jpg', dpi=300)  # Als JPG speichern
# fig.savefig('plot.png', format='png', dpi=300)  # Als PNG speichern

#### 3. Wordcloud:

In [ ]:
# Erstellen einer Wörterbuch-Datenstruktur für die WordCloud
word_freq_dict = dict(top_150_words)

# Erstellen und Anzeigen der WordCloud
wc = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq_dict)

plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')  # Achsen ausblenden
plt.show()

In [ ]:
wc.to_file('wordcloud.png')  # Speichern als PNG-Datei

#### 4. Frequenz eines Wortes pro Jahr

In [ ]:
def tokenize(content): 
    
    wordlist = []
    for word in content.split():
        for char in string.punctuation:
            word = word.strip(char)
        wordlist.append(word) 

    words = [word for word in wordlist if not word.isdecimal()]
    tokens_without_sw = [word for word in words if word not in stopger]

    return tokens_without_sw 

In [ ]:
df['year'] = df['filename'].str[:4]
df['token'] = df['text'].apply(tokenize)
df

In [ ]:
def counting(tokens): 
    counts = Counter(tokens)
    return counts

In [ ]:
df['counts'] = df['token'].apply(counting)
df

In [ ]:
import plotly.express as px

# Wort, dessen Häufigkeit wir analysieren wollen
word_to_analyze = 'Amerika'

# Häufigkeit des Wortes pro Jahr aggregieren
word_counts = df[['year', 'counts']].copy()
word_counts['count'] = word_counts['counts'].apply(lambda x: x.get(word_to_analyze, 0))
word_counts = word_counts.groupby('year')['count'].sum().reset_index()

# Liniendiagramm erstellen
fig = px.line(word_counts, x='year', y='count', title=f'Häufigkeit des Wortes "{word_to_analyze}" pro Jahr', height=550)
fig.update_layout(xaxis_title='Jahr', yaxis_title='Häufigkeit')
fig.show()



In [ ]:
# Anzahl der Einträge pro Jahr zählen
entry_counts = df.groupby('year').size().reset_index(name='entry_count')

# Liniendiagramm erstellen
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=word_counts['year'], y=word_counts['count'], mode='lines', name=f'Häufigkeit des Wortes "{word_to_analyze}"'))
fig2.add_trace(go.Scatter(x=entry_counts['year'], y=entry_counts['entry_count'], mode='lines', name='Anzahl der Einträge'))

fig2.update_layout(
    title=f'Häufigkeit des Wortes "{word_to_analyze}" und Anzahl der Einträge pro Jahr',
    xaxis_title='Jahr',
    yaxis_title='Werte',
    height=550
)

fig2.show()